In [1]:
from pathlib import Path

from autogen_core import CancellationToken
from autogen_core.code_executor import CodeBlock
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

async with DockerCommandLineCodeExecutor(work_dir=work_dir) as executor:
    print(
        await executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code="print('Hello, World!')"),
            ],
            cancellation_token=CancellationToken(),
        )
    )


RuntimeError: Failed to connect to Docker. Please ensure Docker is installed and running.

In [2]:
from pathlib import Path

from autogen_core import CancellationToken
from autogen_core.code_executor import CodeBlock
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

local_executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
print(
    await local_executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code="print('Hello, World!')"),
        ],
        cancellation_token=CancellationToken(),
    )
)


CommandLineCodeResult(exit_code=0, output='Hello, World!\n', code_file='/home/administrator/Desktop/datascience/github/autogen-examples/auto_gen/agents/coding/tmp_code_2b33215fadf3c54926d5c4100348afc158dbff4c94b15044e3a7fe804f80ed2d.py')


In [5]:
import subprocess
from pathlib import Path
import sys
import os

from autogen_core import CancellationToken
from autogen_core.code_executor import CodeBlock
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

venv_dir = work_dir / ".venv"

# Remove existing venv if it exists and is corrupted
if venv_dir.exists():
    import shutil
    shutil.rmtree(venv_dir)

try:
    # Use uv to create virtual environment
    result = subprocess.run(
        ["uv", "venv", str(venv_dir)], 
        capture_output=True, 
        text=True, 
        check=True
    )
    print(f"Virtual environment created successfully with uv: {result.stdout}")
    
    # Set up the virtual environment context manually
    if os.name == 'nt':  # Windows
        python_exe = venv_dir / "Scripts" / "python.exe"
        pip_exe = venv_dir / "Scripts" / "pip.exe"
    else:  # Unix/Linux/macOS
        python_exe = venv_dir / "bin" / "python"
        pip_exe = venv_dir / "bin" / "pip"
    
    # Create a simple virtual environment context
    class UvVenvContext:
        def __init__(self, venv_path):
            self.env_dir = str(venv_path)
            self.env_name = venv_path.name
            if os.name == 'nt':
                self.bin_path = str(venv_path / "Scripts")
                self.env_exe = str(venv_path / "Scripts" / "python.exe")
            else:
                self.bin_path = str(venv_path / "bin")
                self.env_exe = str(venv_path / "bin" / "python")
    
    venv_context = UvVenvContext(venv_dir)
    
except subprocess.CalledProcessError as e:
    print(f"Error creating virtual environment with uv: {e}")
    print(f"stdout: {e.stdout}")
    print(f"stderr: {e.stderr}")
    
    # Fallback: try with standard python venv
    try:
        result = subprocess.run(
            [sys.executable, "-m", "venv", str(venv_dir)], 
            capture_output=True,
            text=True,
            check=True
        )
        print(f"Fallback: Virtual environment created with standard venv: {result.stdout}")
        venv_context = None  # Will use default
    except subprocess.CalledProcessError as fallback_error:
        print(f"Both uv and standard venv failed: {fallback_error}")
        raise
        
except FileNotFoundError:
    print("uv not found. Trying with standard python venv...")
    try:
        result = subprocess.run(
            [sys.executable, "-m", "venv", str(venv_dir)], 
            capture_output=True,
            text=True,
            check=True
        )
        print(f"Virtual environment created with standard venv: {result.stdout}")
        venv_context = None  # Will use default
    except subprocess.CalledProcessError as e:
        print(f"Standard venv also failed: {e}")
        raise

# Create the executor with the virtual environment
if 'venv_context' in locals() and venv_context:
    local_executor = LocalCommandLineCodeExecutor(work_dir=work_dir, virtual_env_context=venv_context)
else:
    local_executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

# Install matplotlib using uv if available, otherwise use pip
try:
    await local_executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="bash", code="uv pip install matplotlib"),
        ],
        cancellation_token=CancellationToken(),
    )
    print("Matplotlib installed successfully with uv")
except Exception as e:
    print(f"uv pip install failed, trying regular pip: {e}")
    await local_executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="bash", code="pip install matplotlib"),
        ],
        cancellation_token=CancellationToken(),
    )
    print("Matplotlib installed with pip")


Virtual environment created successfully with uv: 
Matplotlib installed successfully with uv
Matplotlib installed successfully with uv
